# Solution Optimization Evaluaton Raw TextGrad

In [30]:
import pandas as pd
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifier
from textgrad.loss import TextLoss

## Load Datasets

In [31]:
initial_solution = pd.read_csv("csv/initial_solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,\n Two quantum states with energies E1 and ...,<Solution>\nThe energy-time uncertainty princi...,B,GPQA-Diamond,-
1,2,\n trans-cinnamaldehyde was treated with me...,<Solution>\n1. **Reaction 1:** trans-cinnamald...,A,GPQA-Diamond,-
2,8,\n You perform a high-throughput experiment...,<Solution>\nHere's how we can analyze the resu...,B,GPQA-Diamond,-
3,3,\n A spin-half particle is in a linear supe...,<Solution>\nLet the given state be $|\psi\rang...,D,GPQA-Diamond,-
4,6,\n A quantum mechanical particle of mass m ...,"<Solution>\nThe potential is given by:\nV(r, θ...",D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,404,"\n Under certain conditions, a beam of elec...",<Solution>\nThe de Broglie wavelength of the e...,C,MMLU-CP,college_physics
408,384,\n Two spaceships approach Earth with equal...,<Solution>\nLet $v$ be the speed of each space...,C,MMLU-CP,college_physics
409,411,\n Positronium is an atom formed by an elec...,<Solution>\nThe energy levels of positronium a...,C,MMLU-CP,college_physics
410,396,\n A photon strikes an electron of mass m t...,<Solution>\nLet $E_\gamma$ be the energy of th...,C,MMLU-CP,college_physics


In [32]:
# Test size only 50 rows each datasets (Total 150 rows)

df_gpqa = initial_solution[initial_solution['source'] == 'GPQA-Diamond'].head(50)
df_mmlu_ml = initial_solution[initial_solution['source'] == 'MMLU-ML'].head(50)
df_mmlu_cp = initial_solution[initial_solution['source'] == 'MMLU-CP'].head(50)
df_test = pd.concat([df_gpqa, df_mmlu_ml, df_mmlu_cp], ignore_index=True)

df_test

,id,formatted_question,raw_solution,correct_answer,source,subject
0,1,\n Two quantum states with energies E1 and ...,<Solution>\nThe energy-time uncertainty princi...,B,GPQA-Diamond,-
1,2,\n trans-cinnamaldehyde was treated with me...,<Solution>\n1. **Reaction 1:** trans-cinnamald...,A,GPQA-Diamond,-
2,8,\n You perform a high-throughput experiment...,<Solution>\nHere's how we can analyze the resu...,B,GPQA-Diamond,-
3,3,\n A spin-half particle is in a linear supe...,<Solution>\nLet the given state be $|\psi\rang...,D,GPQA-Diamond,-
4,6,\n A quantum mechanical particle of mass m ...,"<Solution>\nThe potential is given by:\nV(r, θ...",D,GPQA-Diamond,-
...,...,...,...,...,...,...
145,361,\n A grating spectrometer can just barely r...,<Solution>\nThe resolving power R of a grating...,C,MMLU-CP,college_physics
146,339,\n A proton moves in the +z-direction after...,<Solution>\nHere's how to solve this problem:\...,B,MMLU-CP,college_physics
147,364,"\n A net force F_A acts on object A, and a ...",<Solution>\nLet m_A be the mass of object A an...,B,MMLU-CP,college_physics
148,348,"\n The negative muon, mu^-, has properties ...",<Solution>\nA muon (μ) is a fundamental subato...,B,MMLU-CP,college_physics


## Experiment

In [33]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

In [34]:
def evaluate_with_raw_textgrad(row_data):
    match = initial_solution[initial_solution["id"] == row_data["id"]]
    if match.empty:
        return None  # or raise error
    formatted_question = match.iloc[0]["formatted_question"]
    result = {
        "id": row_data["id"],
        "raw_solution": row_data["raw_solution"],
        "correct_answer": row_data["correct_answer"],
        "source": row_data["source"],
        "subject": row_data["subject"]
    }
    
    solution = Variable(row_data["raw_solution"],
                    requires_grad=True,
                    role_description=f"Solution to the math question: {formatted_question}")
    loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                    Do not attempt to solve it yourself, do not give a solution, 
                                    only identify errors. Be super concise. The result still on format
                                    <Solution></Solution><Answer>$LETTER</Answer>""",
                                    requires_grad=False,
                                    role_description="system prompt")
    optimizer = TextualGradientDescent([solution])
    loss = TextLoss(loss_system_prompt, engine=engine)
    
    # Iterate 5 times
    for i in range(1, 6):
        optimizer.zero_grad()  # Clean gradients
        loss_result = loss(solution)
        
        loss_result.backward()
        optimizer.step()
        result[f"solution_{i}"] = solution.value

    return result

## Running Evaluation

### Raw TextGrad

In [35]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

results = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=32) as executor:
    # Submit all tasks
    futures = [
        executor.submit(evaluate_with_raw_textgrad, row.to_dict()) 
        for _, row in df_test.iterrows()
    ]
    
    # Use tqdm for progress tracking
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        result = future.result()
        if result is not None:
            results.append(result)

raw_textgrad = pd.DataFrame(results)

print(f"Completed in {time.time() - start_time:.1f} seconds")
raw_textgrad.to_csv('results/raw_textgrad.csv', index=False)

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. The result still on format\n                                    <Solution></Solution><Answer>$LETTER</Answer> </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nHere\'s how we can analyze the results and determine the relationship between the genes:\n\n1. **G2\'s importance:** The g2 mutant (and any double mutant including g2) shows 0% resistance. This indicates that G2 is essential for resistance to anthracnose.\n\n2. **G1 and G3\'s roles:**  Individually, g1 and g3 mutants show some resistance (75% and 50% respectively). This suggests they contribute to resistance, but aren\'t as crucial as G2.\n\n3. **Interaction between G1 and G3:** The g1g3 double mutant shows drastically reduced resistance (10%) compar

Processing:   0%|          | 0/150 [00:00<?, ?it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. The result still on format\n                                    <Solution></Solution><Answer>$LETTER</Answer> </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nThe potential is given by:\nV(r, θ) = (1/2)kr^2 + (3/2)kr^2 cos^2(θ)\nWe can rewrite this in Cartesian coordinates using x = rcos(θ) and y = rsin(θ), so r^2 = x^2 + y^2.\nV(x, y) = (1/2)k(x^2 + y^2) + (3/2)kx^2\nV(x, y) = (1/2)kx^2 + (3/2)kx^2 + (1/2)ky^2\nV(x, y) = 2kx^2 + (1/2)ky^2\n\nThis potential represents two independent harmonic oscillators in the x and y directions.\nThe potential in the x-direction is V_x = 2kx^2, and the potential in the y-direction is V_y = (1/2)ky^2.\n\nThe angular frequency for a harmonic oscillator is given by ω = sqrt

Processing:   7%|▋         | 11/150 [00:12<02:40,  1.16s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. The result still on format\n                                    <Solution></Solution><Answer>$LETTER</Answer> </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nHere's how to solve this problem:\n\n1. **Conservation of energy:** The total energy before the decay is equal to the total energy after the decay.  Since the pion is at rest, its total energy is its rest mass energy (139.6 MeV).\n\n2. **Energy after decay:** The total energy after the decay is the sum of the energies of the muon and the neutrino. Let's denote the kinetic energy of the muon as KE_mu and the energy of the neutrino as E_nu. Since neutrinos are nearly massless, we can approximate their energy as E_nu = p_nu * c, where p_nu is the moment

...

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. The result still on format\n                                    <Solution></Solution><Answer>$LETTER</Answer> </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nLet m_A and m_B represent the masses of objects A and B, and a_A and a_B represent their respective accelerations. We are given that m_B = 2m_A and a_B = 2a_A.\n\nNewton\'s second law of motion states that force (F) is equal to mass (m) times acceleration (a), or F=ma.  Since force is proportional to both mass and acceleration, doubling both mass and acceleration will quadruple the force.  More specifically, if we double the mass and double the acceleration, the force increases by a factor of 2 * 2 = 4.\n\nTherefore, F_A = m_A * a_A, and F_B = m_B * 

Processing:  99%|█████████▊| 148/150 [03:42<00:03,  1.58s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. The result still on format\n                                    <Solution></Solution><Answer>$LETTER</Answer> </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> <Solution>\nHere's how to solve this problem:\n\n1. **Initial scenario:** The proton is accelerated through a potential difference V and enters a region with crossed electric and magnetic fields. Since its trajectory is unaffected, the electric and magnetic forces must balance. The electric force is F_E = qE in the +x-direction.  Since the proton moves in the +z direction, for the magnetic force to be in the -x direction (to oppose the electric force), the magnetic field *must* be in the -y direction (using the right-hand rule).\n\n2. **Force balance:** The magn

Processing: 100%|██████████| 150/150 [03:53<00:00,  1.56s/it]

Completed in 234.0 seconds
